# 🧊 Spark + Iceberg DML

Iceberg is an open table format for large analytic datasets. It brings ACID transactions, schema evolution, and time travel to data lakes.

## Why Iceberg?
Iceberg enables reliable, scalable, and flexible data management on object storage. It is ideal for the bronze (raw) layer, where data is ingested as-is but must remain queryable, auditable, and mutable.

## 🧩 Roles in the Stack
- **MinIO**: S3-compatible object storage for all table data and metadata files.
- **Hive Metastore**: Stores table and database metadata, enabling Spark and Trino to discover and manage Iceberg tables.

This notebook demonstrates:
- Spark session configuration for Iceberg
- S3/MinIO integration
- Iceberg table creation and DML (CREATE, INSERT, UPDATE)
- Time travel and metadata queries
- Validation and federated access via Trino

🛑 **Prerequisites:** Core services must be running (`docker compose --profile core up -d`).

## ⚙️ Environment Setup

Configure connection URLs and credentials from Docker environment.

In [ ]:
import os

MINIO_ENDPOINT = "http://minio:9000"
MINIO_ACCESS_KEY = os.getenv('MINIO_ROOT_USER', 'minio')
MINIO_SECRET_KEY = os.getenv('MINIO_ROOT_PASSWORD', 'minio123')
HIVE_METASTORE_URI = "thrift://hive-metastore:9083"
TRINO_URL = "http://trino:8080"
SPARK_MASTER = os.getenv('SPARK_MASTER_URL', 'spark://spark-master:7077')

S3_ENDPOINT = "minio:9000"
S3_ACCESS_KEY = MINIO_ACCESS_KEY
S3_SECRET_KEY = MINIO_SECRET_KEY

print("🔧 Environment configured")
print(f"MinIO: {MINIO_ENDPOINT}")
print(f"Hive Metastore: {HIVE_METASTORE_URI}")
print(f"Trino: {TRINO_URL}")
print(f"Spark Master: {SPARK_MASTER}")

os.environ["AWS_REGION"] = "us-east-1"
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

## 🧊 Spark Session with Iceberg

Configure Spark with latest Iceberg JARs and proper catalog settings.

The configuration includes:
- **Iceberg runtime JARs** via `spark.jars.packages`
- **Hive catalog integration** for metadata management  
- **S3/MinIO connectivity** for data storage
- **SQL extensions** for Iceberg operations

In [ ]:
from pyspark.sql import SparkSession
print("🚀 Creating Spark session with Iceberg REST Catalog support...")

packages = [
    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.9.2",
    "org.apache.hadoop:hadoop-aws:3.3.4",
    "software.amazon.awssdk:bundle:2.20.158"
]

try:
    spark = SparkSession.builder \
        .appName("IcebergDMLDemo") \
        .master(SPARK_MASTER) \
        .config("spark.jars.packages", ",".join(packages)) \
        .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
        .config("spark.sql.catalog.spark_catalog.type", "hive") \
        .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
        .config("spark.sql.catalog.iceberg.type", "rest") \
        .config("spark.sql.catalog.iceberg.uri", "http://hive-metastore:9001/iceberg") \
        .config("spark.sql.catalog.iceberg.warehouse", "s3a://iceberg/warehouse") \
        .config("spark.sql.catalog.iceberg.s3.endpoint", f"http://{S3_ENDPOINT}") \
        .config("spark.sql.catalog.iceberg.s3.access-key-id", S3_ACCESS_KEY) \
        .config("spark.sql.catalog.iceberg.s3.secret-access-key", S3_SECRET_KEY) \
        .config("spark.hadoop.fs.s3a.endpoint.region", "us-east-1") \
        .config("spark.sql.catalog.iceberg.s3.region", "us-east-1") \
        .config("spark.executorEnv.AWS_REGION", "us-east-1") \
        .config("spark.executorEnv.AWS_DEFAULT_REGION", "us-east-1") \
        .config("spark.executorEnv.aws.region", "us-east-1") \
        .config("spark.driver.extraJavaOptions", "-Daws.region=us-east-1") \
        .config("spark.executor.extraJavaOptions", "-Daws.region=us-east-1") \
        .config("spark.sql.catalog.iceberg.s3.path-style-access", "true") \
        .config("spark.sql.defaultCatalog", "iceberg") \
        .config("spark.hadoop.hive.metastore.uris", HIVE_METASTORE_URI) \
        .config("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT) \
        .config("spark.hadoop.fs.s3a.access.key", S3_ACCESS_KEY) \
        .config("spark.hadoop.fs.s3a.secret.key", S3_SECRET_KEY) \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
        .enableHiveSupport() \
        .getOrCreate()
    
    print("✅ Spark session created successfully with Iceberg REST Catalog")
    print(f"📊 Spark UI available at: http://localhost:8088")
    print(f"🔧 Spark version: {spark.version}")
    print(f"🌐 Iceberg REST Catalog: http://hive-metastore:9001/iceberg")
    
except Exception as e:
        print(f"❌ All attempts failed: {e}")
        raise

## 🪣 MinIO Bucket Setup

Ensure the iceberg bucket exists in MinIO for Iceberg table storage.

In [ ]:
import boto3
from botocore.exceptions import ClientError

s3_client = boto3.client(
    's3',
    endpoint_url=MINIO_ENDPOINT,
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY
)

def ensure_bucket_exists(bucket_name):
    """Create bucket if it doesn't exist"""
    try:
        s3_client.head_bucket(Bucket=bucket_name)
        print(f"✅ Bucket '{bucket_name}' already exists")
        return True
    except ClientError as e:
        error_code = int(e.response['Error']['Code'])
        if error_code == 404:
            try:
                s3_client.create_bucket(Bucket=bucket_name)
                print(f"✅ Created bucket '{bucket_name}'")
                return True
            except Exception as create_error:
                print(f"❌ Failed to create bucket '{bucket_name}': {create_error}")
                return False
        else:
            print(f"❌ Error checking bucket '{bucket_name}': {e}")
            return False

bucket_ready = ensure_bucket_exists("iceberg")
buckets = s3_client.list_buckets()
print(f"\n📂 Available buckets: {[b['Name'] for b in buckets['Buckets']]}")

## 🔍 Verify Iceberg Integration

Test that Spark can access Iceberg catalogs and the metastore connection works.

In [ ]:
print("🔍 Testing Iceberg catalog integration...")

try:
    catalogs_df = spark.sql("SHOW CATALOGS")
    print("📋 Available Catalogs:")
    catalogs_df.show()
    print("✅ Testing iceberg catalog access...")
    databases_df = spark.sql("SHOW DATABASES IN iceberg")
    print("\n📋 Databases in iceberg catalog:")
    databases_df.show()
    
    print("✅ Iceberg catalog integration verified")
    
except Exception as e:
    print(f"❌ Catalog test failed: {str(e)}")
    print("🔧 This might be expected if no databases exist yet in iceberg catalog")
    try:
        print("🔄 Testing catalog write access...")
        spark.sql("CREATE DATABASE IF NOT EXISTS iceberg.test_connectivity")
        spark.sql("DROP DATABASE IF EXISTS iceberg.test_connectivity")
        print("✅ Catalog write access confirmed")
    except Exception as e2:
        print(f"⚠️  Catalog write test also failed: {str(e2)}")
        print("   Check if Hive Metastore and MinIO are properly connected")

## 🏗️ Create Database and Tables

Create a sample database and demonstrate Iceberg table creation with different patterns.

In [ ]:
database_name = "temp_iceberg_dml"

print(f"🏗️ Creating database: {database_name}")

try:
    print("🔍 Testing catalog access...")
    spark.sql("SHOW DATABASES IN iceberg").show()
    database_location = f"s3a://iceberg/warehouse/{database_name}.db"
    print(f"📍 Database location: {database_location}")
    
    spark.sql(f"CREATE DATABASE IF NOT EXISTS iceberg.{database_name} LOCATION '{database_location}'")
    print(f"✅ Database '{database_name}' created successfully")
    spark.sql(f"USE iceberg.{database_name}")
    current_db = spark.sql("SELECT current_database()").collect()[0][0]
    print(f"📍 Current database: {current_db}")
    print("\n📋 Available databases:")
    spark.sql("SHOW DATABASES IN iceberg").show()
    
except Exception as e:
    print(f"❌ Database creation failed: {str(e)}")
    print("\n🔧 Troubleshooting suggestions:")
    print("   1. Check if Hive Metastore is running and accessible")
    print("   2. Verify MinIO bucket 'iceberg' exists and is accessible")
    print("   3. Check S3A filesystem configuration")

In [ ]:
print("🧊 Creating Iceberg table: customers")
customers_ddl = """
CREATE TABLE IF NOT EXISTS customers (
    customer_id BIGINT,
    first_name STRING,
    last_name STRING,
    email STRING,
    registration_date DATE,
    last_login TIMESTAMP,
    lifetime_value DOUBLE,
    segment STRING
) USING iceberg
PARTITIONED BY (segment)
TBLPROPERTIES (
    'write.target-file-size-bytes'='134217728',
    'write.parquet.compression-codec'='snappy'
)
"""

try:
    spark.sql(customers_ddl)
    print("✅ Customers table created successfully with Iceberg format")
    spark.sql("DESCRIBE EXTENDED customers").show(20, truncate=False)
    
except Exception as e:
    print(f"❌ Failed to create Iceberg table: {str(e)}")
    print("\n🔧 This error suggests S3A/MinIO connectivity issues")
    print(f"❌ All approaches failed: {str(e3)}")
    print("\n💡 Next steps:")
    print("   1. Ensure docker compose --profile core is running")
    print("   2. Check MinIO console at http://localhost:9001")
    print("   3. Verify Hive Metastore logs for connection issues")
    raise

In [ ]:
print("🧊 Creating Iceberg table: orders")

orders_ddl = """
CREATE TABLE IF NOT EXISTS orders (
    order_id BIGINT,
    customer_id BIGINT,
    order_date DATE,
    order_timestamp TIMESTAMP,
    total_amount DOUBLE,
    status STRING,
    payment_method STRING,
    shipping_address STRING
) USING iceberg
PARTITIONED BY (months(order_date))
TBLPROPERTIES (
    'write.target-file-size-bytes'='268435456',
    'write.parquet.compression-codec'='zstd'
)
"""

try:
    spark.sql(orders_ddl)
    print("✅ Orders table created successfully")
except Exception as e:
    print(f"❌ Failed to create orders table: {str(e)}")
    raise

## 📝 Insert Sample Data

Insert realistic sample data into our Iceberg tables using Spark SQL DML.

### 📝 DML Operations with Iceberg

Iceberg supports full ACID DML: CREATE, INSERT, UPDATE, DELETE. Each operation is atomic and tracked in table metadata.

In [ ]:
from datetime import datetime, date, timedelta
import random

print("📝 Inserting sample customer data...")
customer_data = []
segments = ['Premium', 'Standard', 'Basic']
domains = ['gmail.com', 'yahoo.com', 'company.com', 'outlook.com']

for i in range(1, 101):
    customer_data.append((
        i,
        f"Customer_{i:03d}",
        f"Last_{i:03d}",
        f"customer_{i:03d}@{random.choice(domains)}",
        date(2022, 1, 1) + timedelta(days=random.randint(0, 1000)),
        datetime.now() - timedelta(hours=random.randint(1, 8760)),
        round(random.uniform(100.0, 10000.0), 2),
        random.choice(segments)
    ))

customers_df = spark.createDataFrame(
    customer_data,
    ["customer_id", "first_name", "last_name", "email", "registration_date", "last_login", "lifetime_value", "segment"]
)

customers_df.writeTo("customers").append()
print("✅ Customer data inserted successfully")

customer_count = spark.sql("SELECT COUNT(*) as count FROM customers").collect()[0]['count']
print(f"📊 Total customers: {customer_count}")

In [ ]:
print("📝 Inserting sample order data...")

order_data = []
statuses = ['Pending', 'Processing', 'Shipped', 'Delivered', 'Cancelled']
payment_methods = ['Credit Card', 'PayPal', 'Bank Transfer', 'Cash']

for i in range(1, 301):
    order_date = date(2024, 1, 1) + timedelta(days=random.randint(0, 250))
    order_data.append((
        i,
        random.randint(1, 100),
        order_date,
        datetime.combine(order_date, datetime.min.time()) + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59)),
        round(random.uniform(25.0, 2500.0), 2),
        random.choice(statuses),
        random.choice(payment_methods),
        f"Address {i}, City, State {random.randint(10000, 99999)}"
    ))

orders_df = spark.createDataFrame(
    order_data,
    ["order_id", "customer_id", "order_date", "order_timestamp", "total_amount", "status", "payment_method", "shipping_address"]
)

orders_df.writeTo("orders").append()
print("✅ Order data inserted successfully")

order_count = spark.sql("SELECT COUNT(*) as count FROM orders").collect()[0]['count']
print(f"📊 Total orders: {order_count}")

## 🔍 Query and Analyze Data

Demonstrate various Iceberg features through SQL queries.

In [ ]:
print("📊 Running analytics queries on Iceberg tables...")
print("\n🎯 Customer Segment Analysis:")
segment_analysis = spark.sql("""
    SELECT 
        segment,
        COUNT(*) as customer_count,
        AVG(lifetime_value) as avg_lifetime_value,
        MAX(lifetime_value) as max_lifetime_value
    FROM customers 
    GROUP BY segment 
    ORDER BY avg_lifetime_value DESC
""")
segment_analysis.show()

In [ ]:
print("📅 Monthly Order Trends:")
monthly_trends = spark.sql("""
    SELECT 
        YEAR(order_date) as year,
        MONTH(order_date) as month,
        COUNT(*) as order_count,
        SUM(total_amount) as total_revenue,
        AVG(total_amount) as avg_order_value
    FROM orders 
    GROUP BY YEAR(order_date), MONTH(order_date)
    ORDER BY year, month
""")
monthly_trends.show()

In [ ]:
print("🔗 Customer-Order Join Analysis:")
customer_order_analysis = spark.sql("""
    SELECT 
        c.segment,
        c.first_name,
        c.last_name,
        COUNT(o.order_id) as total_orders,
        SUM(o.total_amount) as total_spent,
        AVG(o.total_amount) as avg_order_value
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.segment, c.first_name, c.last_name
    HAVING total_orders > 0
    ORDER BY total_spent DESC
    LIMIT 10
""")
customer_order_analysis.show()

## 🕰️ Iceberg Time Travel

Demonstrate Iceberg's time travel capabilities by viewing table snapshots and history.

In [ ]:
print("🕰️ Iceberg Time Travel Features")
print("\n📸 Table Snapshots for customers:")
try:
    snapshots = spark.sql("SELECT * FROM iceberg.temp_iceberg_dml.customers.snapshots")
    snapshots.select("committed_at", "snapshot_id", "operation", "summary").show(truncate=False)
except Exception as e:
    print(f"Snapshots query failed: {e}")

print("\n📜 Table History for customers:")
try:
    history = spark.sql("SELECT * FROM iceberg.temp_iceberg_dml.customers.history")
    history.show(truncate=False)
except Exception as e:
    print(f"History query failed: {e}")

In [ ]:
print("🔄 Performing UPDATE operation for time travel demo...")
update_sql = """
    UPDATE customers 
    SET lifetime_value = lifetime_value * 1.1,
        segment = CASE 
            WHEN lifetime_value > 5000 THEN 'Premium'
            WHEN lifetime_value > 1000 THEN 'Standard'
            ELSE 'Basic'
        END
    WHERE segment = 'Premium'
"""

try:
    spark.sql(update_sql)
    print("✅ Update operation completed")
    updated_counts = spark.sql("""
        SELECT segment, COUNT(*) as count 
        FROM customers 
        GROUP BY segment 
        ORDER BY segment
    """)
    print("\n📊 Updated segment distribution:")
    updated_counts.show()
except Exception as e:
    print(f"❌ Update failed: {str(e)}")

## 🔗 Trino Integration Verification

Verify that our Iceberg tables are accessible through Trino for federated queries.

## 🔗 Why Validate and Access via Trino?

Trino is a distributed SQL query engine that can access Iceberg tables directly via the Hive Metastore. This allows us to:
- Validate that tables are discoverable and queryable outside Spark.
- Enable federated analytics across multiple data sources.
- Provide a single SQL interface for BI tools and analysts.

Using Trino ensures our Iceberg tables are interoperable and production-ready.

In [ ]:
print("🔗 Testing Trino integration with Iceberg tables...")

try:
    from trino.dbapi import connect as trino_connect
    import pandas as pd
    
    trino_conn = trino_connect(
        host='trino',
        port=8080,
        user='admin',
        catalog='iceberg',
        schema='temp_iceberg_dml'
    )
    
    def query_trino(sql):
        cursor = trino_conn.cursor()
        cursor.execute(sql)
        columns = [desc[0] for desc in cursor.description]
        data = cursor.fetchall()
        return pd.DataFrame(data, columns=columns)
    print("📋 Available schemas in Trino iceberg catalog:")
    schemas = query_trino("SHOW SCHEMAS")
    print(schemas)
    if 'temp_iceberg_dml' in schemas['Schema'].values:
        print("\n📋 Tables in temp_iceberg_dml schema via Trino:")
        tables = query_trino("SHOW TABLES")
        print(tables)
        print("\n📊 Sample data via Trino:")
        sample_data = query_trino("SELECT segment, COUNT(*) as count FROM customers GROUP BY segment")
        print(sample_data)
        
        print("✅ Trino integration verified successfully!")
    else:
        print("⚠️ temp_iceberg_dml schema not yet visible in Trino")
        
except ImportError:
    print("📦 Trino client not available, skipping integration test")
except Exception as e:
    print(f"🔗 Trino integration test failed: {str(e)}")
    print("   This might be expected if Trino catalog refresh is needed")

## 🧹 Table Maintenance

Demonstrate Iceberg table maintenance operations like OPTIMIZE and EXPIRE SNAPSHOTS.

In [ ]:
print("🧹 Performing table maintenance operations...")
print("📂 Table files before optimization:")
try:
    files_before = spark.sql("SELECT * FROM iceberg.temp_iceberg_dml.customers.files")
    file_count_before = files_before.count()
    print(f"   Files count: {file_count_before}")
    if file_count_before > 0:
        files_before.select("file_size_in_bytes").describe().show()
except Exception as e:
    print(f"   Files query failed: {e}")
print("\n⚡ Optimizing customers table...")
try:
    optimize_result = spark.sql("CALL iceberg.system.rewrite_data_files('temp_iceberg_dml.customers')")
    optimize_result.show()
    print("✅ Table optimization completed")
except Exception as e:
    print(f"❌ Optimization failed: {str(e)}")

In [ ]:
print("🗑️ Cleaning up old snapshots...")

try:
    expire_result = spark.sql(f"""
        CALL iceberg.system.expire_snapshots(
            table => 'temp_iceberg_dml.customers',
            older_than => TIMESTAMP '{datetime.now() - timedelta(hours=1)}'
        )
    """)
    expire_result.show()
    print("✅ Snapshot cleanup completed")
except Exception as e:
    print(f"❌ Snapshot cleanup failed: {str(e)}")
    print("   This might be expected if no old snapshots exist")

## 📊 Performance and Metadata Analysis

Analyze table metadata and performance characteristics.

In [ ]:
print("📊 Table Metadata Analysis")
tables_to_analyze = ['customers', 'orders']

for table_name in tables_to_analyze:
    print(f"\n🔍 Analyzing table: {table_name}")
    
    try:
        describe_result = spark.sql(f"DESCRIBE EXTENDED iceberg.temp_iceberg_dml.{table_name}")
        print(f"   Schema and properties:")
        describe_result.show(20, truncate=False)
            
    except Exception as e:
        print(f"   Analysis failed: {str(e)}")

## 📝 Summary

- Iceberg brings reliability and flexibility to data lakes.
- MinIO stores the data; Hive Metastore tracks the metadata.
- Spark and Trino both access Iceberg tables for analytics and validation.
- All basic DML operations are supported with ACID guarantees.

For more, see the project README and docs/guidelines.md.

In [ ]:
print("🎯 Spark + Iceberg Integration Summary")
print("=" * 50)

try:
    customer_count = spark.sql("SELECT COUNT(*) FROM iceberg.temp_iceberg_dml.customers").collect()[0][0]
    order_count = spark.sql("SELECT COUNT(*) FROM iceberg.temp_iceberg_dml.orders").collect()[0][0]
    
    print(f"✅ Successfully created Iceberg tables:")
    print(f"   📊 Customers: {customer_count:,} records")
    print(f"   📊 Orders: {order_count:,} records")
    print(f"\n📂 Database structure:")
    tables = spark.sql("SHOW TABLES IN iceberg.temp_iceberg_dml")
    for row in tables.collect():
        print(f"   🧊 {row['tableName']} (iceberg)")
    
    print(f"\n🔧 Key configurations demonstrated:")
    print(f"   📦 JAR management via spark.jars.packages")
    print(f"   🗂️ Hive Metastore integration")
    print(f"   🪣 MinIO/S3 storage backend")
    print(f"   🕰️ Time travel capabilities")
    print(f"   🧹 Table maintenance operations")
    print(f"   🔗 Trino federated query access")
    
    print(f"\n🚀 Next steps to explore:")
    print(f"   📈 Schema evolution with ALTER TABLE")
    print(f"   🔄 Streaming ingestion with Structured Streaming")
    print(f"   📊 Advanced analytics with window functions")
    print(f"   🏗️ Data iceberg patterns with Delta/Iceberg")
    print(f"   🔍 Query optimization and performance tuning")
    
except Exception as e:
    print(f"❌ Summary generation failed: {str(e)}")

print(f"\n🎉 Iceberg integration complete!")

## 🧹 Cleanup: Drop Tables and Database

The following cell will remove all demo tables and the database created during this notebook. Use with caution if you want to keep your data.

In [ ]:
try:
    print("🧹 Dropping tables and database...")
    spark.sql("DROP TABLE IF EXISTS customers")
    spark.sql("DROP TABLE IF EXISTS orders")
    spark.sql("DROP DATABASE IF EXISTS temp_iceberg_dml")
    print("✅ Cleanup complete: tables and database dropped.")
except Exception as e:
    print(f"❌ Cleanup failed: {e}")